# Testing of generative AI

In [6]:
# imports
import requests
import json

In [4]:
def generateCode(strPrompt):
    url = 'http://deeperthought.cse.chalmers.se:5050/api/generate'
    
    data = {
        "model": "llama3.2",
        "prompt": strPrompt,
        "stream": False
    }
    
    headers = {'Content-Type': 'application/json'}

    response = requests.post(url, data=json.dumps(data), headers=headers)

    json_data = json.loads(response.text)

    strProgram = json_data['response']

    return strProgram

In [ ]:
from datasets import load_dataset

# Load the glue dataset
dataset = load_dataset('glue', 'mrpc')

# Apply your function to the dataset
def apply_function_to_dataset(dataset, function):
    return dataset.map(lambda examples: {'result': function(examples['sentence1'])})

# Apply the function to the train split
result_dataset = apply_function_to_dataset(dataset['train'].select(range(50)), generateCode)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [19]:
# Evaluate the results (example: print the first few results)
print(result_dataset[:5])

{'sentence1': ['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .", 'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .', 'Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .', 'The stock rose $ 2.11 , or about 11 percent , to close Friday at $ 21.51 on the New York Stock Exchange .'], 'sentence2': ['Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .", "On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale .", 'Tab shares jumped 20 cents , or 4.6 % , to set a record closing high at A $ 4.57 .', 'P

In [18]:
import sacrebleu

# Extract references and predictions
references = [example['sentence2'] for example in dataset['train']]
predictions = [example['result'] for example in result_dataset]

# Calculate BLEU score
bleu = sacrebleu.corpus_bleu(predictions, [references])
print(f"BLEU score: {bleu.score:.2f}")

BLEU score: 1.74
